In [1]:
import numpy as np 
import pandas as pd

In [10]:
#와이드 포맷 읽기
df_wide= pd.read_csv('input/ch03/time_series_wide.csv', index_col= 0)
df_wide.head()

,A,B,C
2016-07-01,532,3314,1136
2016-07-02,798,2461,1188
2016-07-03,823,3522,1711
2016-07-04,937,5451,1977
2016-07-05,881,4729,1975


In [11]:
#인덱스 형태를 날짜형으로 변경
df_wide.index= pd.to_datetime(df_wide.index)

In [12]:
print(df_wide.iloc[:5, :3])

              A     B     C
2016-07-01  532  3314  1136
2016-07-02  798  2461  1188
2016-07-03  823  3522  1711
2016-07-04  937  5451  1977
2016-07-05  881  4729  1975


In [13]:
#롱 포맷으로 변환 
df_long = df_wide.stack().reset_index(1)

In [14]:
df_long.columns = ['id', 'value']

In [15]:
print(df_long.head(10))

           id  value
2016-07-01  A    532
2016-07-01  B   3314
2016-07-01  C   1136
2016-07-02  A    798
2016-07-02  B   2461
2016-07-02  C   1188
2016-07-03  A    823
2016-07-03  B   3522
2016-07-03  C   1711
2016-07-04  A    937


In [16]:
#롱 포맷을 와이드 포맷으로 되돌림 
df_wide= df_long.pivot(index=None, columns='id', values='value')

In [17]:
x= df_wide

In [18]:
x_lag1= x.shift(1)
x_lag7=x.shift(7)

In [19]:
x_lag1

id,A,B,C
2016-07-01,NaN,NaN,NaN
2016-07-02,532.0,3314.0,1136.0
2016-07-03,798.0,2461.0,1188.0
2016-07-04,823.0,3522.0,1711.0
2016-07-05,937.0,5451.0,1977.0
...,...,...,...
2016-12-27,840.0,4576.0,1954.0
2016-12-28,840.0,4573.0,1850.0
2016-12-29,943.0,4511.0,1764.0
2016-12-30,978.0,4599.0,1787.0


In [20]:
# 3기간의 이동평균 
x_avg3=x.shift(1).rolling(window=3).mean()

In [21]:
# 7기간 범위에서의 최대값 
x_max7=x.shift(1).rolling(window=7).max()

In [22]:
# 7일, 14일, 21일, 28일 이전의 합의 평균으로 치환 
x_e7_avg=(x.shift(7)+x.shift(14)+x.shift(21)+x.shift(28))/4.0

In [23]:
# 1일 이후의 값 
x_lead1=x.shift(-1)

In [24]:
#lag 특징 

train_x = pd.read_csv('input/ch03/time_series_train.csv')
event_history= pd.read_csv('input/ch03/time_series_events.csv')
train_x['date']=pd.to_datetime(train_x['date'])
event_history['date']=pd.to_datetime(event_history['date'])

In [25]:
train_x

,user_id,date,target
0,1,2018-01-01,1
1,1,2018-01-02,1
2,1,2018-01-03,1
3,1,2018-01-04,1
4,1,2018-01-05,0
...,...,...,...
36495,100,2018-12-27,0
36496,100,2018-12-28,1
36497,100,2018-12-29,1
36498,100,2018-12-30,1


In [27]:
event_history

,date,event
0,2018-01-03,sale
1,2018-01-03,conpon
2,2018-01-04,points
3,2018-01-05,points
4,2018-05-03,sale
5,2018-05-04,sale
6,2018-05-05,sale
7,2018-05-06,points
8,2018-05-07,points
9,2018-05-08,points


In [30]:
dates=np.sort(train_x['date'].unique())
occurrences= pd.DataFrame(dates, columns=['date'])

In [31]:
occurrences

,date
0,2018-01-01
1,2018-01-02
2,2018-01-03
3,2018-01-04
4,2018-01-05
...,...
360,2018-12-27
361,2018-12-28
362,2018-12-29
363,2018-12-30


In [32]:
sale_history=event_history[event_history['event']=='sale']
occurrences['sale']= occurrences['date'].isin(sale_history['date'])

In [36]:
occurrences['sale']=occurrences['sale'].cumsum()

In [37]:
train_x= train_x.merge(occurrences, on='date', how='left')

In [38]:
train_x

,user_id,date,target,sale
0,1,2018-01-01,1,0
1,1,2018-01-02,1,0
2,1,2018-01-03,1,1
3,1,2018-01-04,1,1
4,1,2018-01-05,0,1
...,...,...,...,...
36495,100,2018-12-27,0,4
36496,100,2018-12-28,1,4
36497,100,2018-12-29,1,4
36498,100,2018-12-30,1,5
